# Karatsuba Algorithm for Multiplying Two Polynomials
## Sriram Sankaranarayanan
### CSCI 3104 Algorithms


## Representing Polynomials

We will use an array of floats of the form [a[0], a[1], ..., a[n-1]] to represent the polynomial $a_0 + a_1 x + a_2 x^2 + \cdots + a_{n-1}x^{n-1}$.

In [11]:
from sympy.interactive import printing
printing.init_printing(use_latex=True)

def prettyPrintPolynomial(a):
    n = len(a)
    rstr=''
    print('$',end='')
    for (i,e) in enumerate(a):
        if ( e > 0):
            rstr += '+'
            print('+',end='')
        if (e != 0):
            rstr += str(e)
            print(e,end='')
            if (i ==1):
                rstr += 'x'
                print('*x',end='')
            if (i > 1):
                rstr += 'x^%d'%(i)
                print('*x^%d'%(i),end='')
    print('$')
    return rstr

In [12]:
from IPython.core.display import display, Math, Latex

p1=[1,-2,0,3,4,0,0,0,7]
display(Math(prettyPrintPolynomial(p1)))
p2=[1,2,3,0,0,0,0,0,1,1]
display(Math(prettyPrintPolynomial(p2)))

$+1-2*x+3*x^3+4*x^4+7*x^8$


<IPython.core.display.Math object>

$+1+2*x+3*x^2+1*x^8+1*x^9$


<IPython.core.display.Math object>

# Basic operations on polynomials

In [21]:
# addition of two polynomials
def scaleAndAdd(a,b,s): # compute c(x) = a(x) + s * b(x) where s is scalar
    m = len(a)
    n = len(b)
    k = min(m,n)
    c=[]
    for i in range(0,k):
        c.append(a[i]+s*b[i])
    for j in range(k,m):
        c.append(a[j])
    for l in range(k,n):
        c.append(s*b[l])
    return c

def scaleAndShift(p,s,k): # compute s*x^k *p(x)
    q=[]
    n = len(p)
    for j in range(0,k):
        q.append(0)
    for i in range(0,n):
        q.append(p[i]*s)
    #q = s * p
    assert(len(q)== n+k)
    return q

def shift(p,k):
    return scaleAndShift(p,1.0,k)

def scale(p,s):
    return scaleAndShift(p,s,0)

def add(a,b):
    return scaleAndAdd(a,b,1.0)

def sub(a,b):
    return scaleAndAdd(a,b,-1.0)

In [4]:
p3=add(p1,p2)
prettyPrintPolynomial(p3)
p4 = sub(p1,p2)
prettyPrintPolynomial(p4)
p5 = scaleAndShift(p4,2.0,5)
prettyPrintPolynomial(p5)

$+2.0+3.0*x^2+3.0*x^3+4.0*x^4+8.0*x^8+1.0*x^9$
$-4.0*x-3.0*x^2+3.0*x^3+4.0*x^4+6.0*x^8-1.0*x^9$
$-8.0*x^6-6.0*x^7+6.0*x^8+8.0*x^9+12.0*x^13-2.0*x^14$


## Multiplication (Grade School)

In [5]:
def gradeSchoolMultiply(p, q):
    # the algorithm simply goes over 
    # each term of one polynomial p and uses it to 
    # scale q
    m = len(p)
    n = len(q)
    if (m == 0 or n == 0):
        return []
    res = []
    for (i,c) in enumerate(p):     # this loop runs m times
        r1 = scaleAndShift(q,c,i)  # this takes O(n) time
        res = add(res,r1)          # this takes O(n+m) time
    return res   
        

In [6]:
p6 = gradeSchoolMultiply(p1,p2)
prettyPrintPolynomial(p6)

$+1.0-1.0*x^2-3.0*x^3+10.0*x^4+17.0*x^5+12.0*x^6+8.0*x^8+13.0*x^9+19.0*x^10+3.0*x^11+7.0*x^12+4.0*x^13+7.0*x^16+7.0*x^17$


In [7]:
a=[0,1,2] # x  +2 x^2
b=[1,0,2] # 1 + 2x^2
c = gradeSchoolMultiply(a,b)
prettyPrintPolynomial(c)

$+1.0*x+2.0*x^2+2.0*x^3+4.0*x^4$


## Analysis of Grade School Multiplication

Multiplying a polynomial $p$ of degree $m$ with a polynomial $q$ of degree $n$ involves $m$ calls to the scaleAndShift routine and $m$ calls to add routine each taking $O(n+m)$ time. The overall algorithm takes time $O((m+n)^2)$. If $n = m$ this is $O(n^2)$. I am using O(.) rather than $\Theta(.)$ because my analysis is not precise to pinpoint the exact number of operations for each loop iteration. That is not too hard but it is tedious.

## Karatsuba's Algorithm

Let $p(x) = a_0 + a_1 x + \cdots + a_{n-1} x^{n-1}$ and $q(x) = b_0 + b_1 x + \cdots + b_{m-1} x^{m-1}$. 

Let us split $p$ into two roughly equal parts:

- $p_1$ of degree $k = \lfloor \frac{n}{2} \rfloor $ and $p_2$. 
- We can write $p = p_1 + x^{k+1} p_2$.

Likewise, let us write $q = q_1 + x^{l+1}q_2$. In general, $k \not= l$.

The product is
$$ \begin{array}{rcl}
p * q &=& p_1 * q_1 + x^{k+1} p_1 * q_2 + x^{l+1}* p_2 * q_1 + x^{k+l+2} * p_2 * q_2 \\
&=& \underset{r_1}{\underbrace{p_1 * q_1}} + x^j ( \underset{r}{\underbrace{x^{k+1-j} p_1 * q_2 + x^{l+1-j} *p_2*q_1}}) + x^{k+l+2} * \underset{r_2}{\underbrace{p_2 * q_2}}\\
\end{array}
$$

Here $j=\min(k+1,l+1)$

To compute this product, we compute
$$r_1 = p_1 * q_1\ \mbox{and}\ r_2 = p_2 * q_2 $$

Next we compute $$ r_3 = (x^{l+1-j}*p_1 + p_2) * (x^{k+1-j}*q_1 + q_2)$$

We can obtain $$ r = r_3 - x^{k+l+2} r_1 - r_2 $$

Multiplying a polynomial $p$ by $x^j$ is achieved by shifting the array and padding $j$ initial $0$s. The divide and conquer algorithm makes recursive calls to itself.


In [40]:
def karatsubaMultiply(p,q):
    m = len(p)
    n = len(q)
    if (m==0 or n == 0):
        return []
    if (m <= 4 or n <= 4):
        return gradeSchoolMultiply(p,q)
    k= int(m/2)
    l= int(n/2)
    j= min(k+1,l+1)
    
    # Split the arrays
    
    p1 = p[0:k+1]
    p2 = p[k+1:m]
    q1 = q[0:l+1]
    q2 = q[l+1:n]
    assert(len(p1)==k+1)
    assert(len(q1)==l+1)
    
    #Recursive multiply
    r1 = karatsubaMultiply(p1,q1) # recursive call to compute r1 = p1*q1
    
    r2 = karatsubaMultiply(p2,q2) # r2 = p2*q2 using recursive call number 2
    
    s1 = shift(p1,l+1-j)
    s = add(s1,p2) # s = p1*x^{l+1-j} + p2
    t1 = shift(q1,k+1-j)
    t = add(t1,q2) # t = q1*x^{k+1-j}+ q2
    r3 = karatsubaMultiply(s,t) # r3 = s  * t recursive call number 3
    
    # the rest of the calculation
    r4 = sub(r3,r2)            #r4 = r3 - p2*q2  
    s2 = shift(r1,k+l+2-2*j)   #s2 = p1*q1*x^{k+l+2-2j}
    r50 = sub(r4,s2)           #r50 = r4-s2
    r5 = shift(r50,j)          # r5 = r50*x^j
    # Now put the final result together
    r12 = add(r1,r5)
    r6 = shift(r2,k+l+2)
    res = add(r12,r6)
    return res    

In [26]:
p61=karatsubaMultiply(p1,p6) # karatsuba
prettyPrintPolynomial(p61)
p62=gradeSchoolMultiply(p1,p6) # vs gradeschool
d = sub(p61,p62)
prettyPrintPolynomial(d)

$+1.0-2.0*x-1.0*x^2+2.0*x^3+20.0*x^4-6.0*x^5-35.0*x^6-6.0*x^7+106.0*x^8+101.0*x^9+34.0*x^10-32.0*x^11+142.0*x^12+218.0*x^13+161.0*x^14+33.0*x^15+103.0*x^16+100.0*x^17+119.0*x^18+42.0*x^19+98.0*x^20+56.0*x^21+49.0*x^24+49.0*x^25$
$$


In [44]:
import random
def testKarat(n,nTrials):  # Compare both multiplications on random polynomials
    for t in range(0,nTrials):
        m = random.randint(int(n/2),n)
        a = [random.randint(0,10) for i in range(0,m)]
        b = [random.randint(0,10) for i in range(0,n)]
        p1 = gradeSchoolMultiply(a,b)
        p2 = karatsubaMultiply(a,b)
        d0 = sub(p1,p2)
        # Check if the difference is all 0 by looking for nonzero coefficients
        # of the difference polynomial
        d = [i for (i,e) in enumerate(d0) if e != 0.0]
        if (len(d) > 0): # There is a nonzero coefficient
            print('FAILED:')
            print(a)
            print(b)
            print(d)
            assert(False)
    # Otherwise, we ran through all tests without finding a discrepancy
    print('Success')

In [43]:
testKarat(30,1000)
testKarat(50,1000)

Success
Success


## Complexity Analysis

Let $T(n)$ be the number of steps needed to multiply two polynomials, both of size $n$ in the worst case using Karatsuba.

We have that each run then generates three recursive calls each of size $\frac{n}{2}$ and furthermore, involves a bunch of additions, subtractions and shifts, all of which are $\Theta(n)$.

Therefore,

$$ T(n) = \begin{cases} 
 3 T\left(\frac{n}{2}\right) + \Theta(n) & n > 6 \\
  C & n \leq 6 \\
  \end{cases}$$
  
  This recurrence can be solved by master method and yields $T(n) = \Theta(n^{\log_2(3)}) = \Theta(n^{1.54...})$.
  Thus the Karatsuba algorithm is asymptotically faster than grade school